In [69]:
import pandas as pd
import pdb

def get_teams_season_past_matches(filepath, team, season, date):
    matches = pd.read_csv(filepath)
    seasons_matches = matches[matches.season_id == season]
    past_matches = seasons_matches[seasons_matches.date < date]

    home_team_matches = past_matches[past_matches.home_team_name == team]
    
    # TODO: calculate win's, losses, ties, results for team and opponent

    home_team_matches = home_team_matches.drop('home_team_name', axis=1)
    home_team_matches = home_team_matches.drop('away_team_name', axis=1)
    home_team_matches = home_team_matches.drop('match_id', axis=1)
    home_team_matches = home_team_matches.drop('competition_id', axis=1)
    home_team_matches = home_team_matches.drop('season_id', axis=1)
    home_team_matches = home_team_matches.drop('date', axis=1)
    
    sum_home_team_matches = home_team_matches.sum()
    sum_home_team_matches['team_score'] = sum_home_team_matches['score_home']
    sum_home_team_matches['opponent_score'] = sum_home_team_matches['score_away']
    home_num_matches = len(home_team_matches)
        
    away_team_matches = past_matches[past_matches.away_team_name == team]

    away_team_matches = away_team_matches.drop('home_team_name', axis=1)
    away_team_matches = away_team_matches.drop('away_team_name', axis=1)
    away_team_matches = away_team_matches.drop('match_id', axis=1)
    away_team_matches = away_team_matches.drop('competition_id', axis=1)
    away_team_matches = away_team_matches.drop('season_id', axis=1)
    away_team_matches = away_team_matches.drop('date', axis=1)
    
    sum_away_team_matches = away_team_matches.sum()
    sum_away_team_matches['team_score'] = sum_away_team_matches['score_away']
    sum_away_team_matches['opponent_score'] = sum_away_team_matches['score_home']
    away_num_matches = len(away_team_matches)

    if home_num_matches == 0 and away_num_matches == 0:
        return None
    return (sum_home_team_matches + sum_away_team_matches) / float(home_num_matches + away_num_matches)

In [73]:
def add_metadata(df, match):
    if match.score_home > match.score_away:
        outcome = 'H'
        home_result = True
        away_result = False
    elif match.score_home == match.score_away:
        outcome = 'D'
        home_result = True
        away_result = True
    else:
        outcome = 'A'
        home_result = False
        away_result = True
    df['outcome'] = outcome
    df['home_result'] = home_result
    df['away_result'] = away_result
    df['date'] = match.date
    df['match_id'] = match.match_id
    df['home_team_name'] = match.home_team_name
    df['away_team_name'] = match.away_team_name
    df['competition_id'] = match.competition_id
    df['season_id'] = match.season_id
    df['score_home'] = match.score_home
    df['score_away'] = match.score_away
    return df

filepath = 'data/matches.csv'
matches = pd.read_csv(filepath)
# TODO: Filter matches here if we only want certain leagues or seasons...
res_exists = False

for i, match in matches.iterrows():
    if i % 100 == 0:
        print(i, 'of', len(matches))
    home_stats = get_teams_season_past_matches(filepath, match.home_team_name, match.season_id, match.date)
    away_stats = get_teams_season_past_matches(filepath, match.away_team_name, match.season_id, match.date)
    if not res_exists:
        if home_stats is not None and away_stats is not None:
            home_res = home_stats.to_frame().transpose().add_prefix('home_')
            away_res = away_stats.to_frame().transpose().add_prefix('away_')
            res_exists = True
            res = pd.concat([home_res, away_res], axis=1)
            res = add_metadata(res, match)
    else:
        if home_stats is not None and away_stats is not None:
            home_res = home_stats.to_frame().transpose().add_prefix('home_')
            away_res = away_stats.to_frame().transpose().add_prefix('away_')
            temp = pd.concat([home_res, away_res], axis=1)
            temp = add_metadata(temp, match)
            res = res.append(temp)

0 of 3804
100 of 3804
200 of 3804
300 of 3804
400 of 3804
500 of 3804
600 of 3804
700 of 3804
800 of 3804
900 of 3804
1000 of 3804
1100 of 3804
1200 of 3804
1300 of 3804
1400 of 3804
1500 of 3804
1600 of 3804
1700 of 3804
1800 of 3804
1900 of 3804
2000 of 3804
2100 of 3804
2200 of 3804
2300 of 3804
2400 of 3804
2500 of 3804
2600 of 3804
2700 of 3804
2800 of 3804
2900 of 3804
3000 of 3804
3100 of 3804
3200 of 3804
3300 of 3804
3400 of 3804
3500 of 3804
3600 of 3804
3700 of 3804
3800 of 3804


In [84]:
res[res.away_team_name == 'Real Salt Lake'].shape

(41, 395)

In [85]:
res.to_csv('data/past_matches.csv')

In [71]:
# pd.set_option('display.max_columns', None) # None for all, 0 for screen width
# match
# res

,home_score_home,home_score_away,home_half_score_home,home_half_score_away,home_away_equalizer_goal,home_away_matchwinner_goal,home_away_outofplayconceded_f24,home_away_savesbody_f24,home_away_savescaught_f24,home_away_savescollected_f24,home_away_savesdiving_f24,home_away_savesfeet_f24,home_away_savesfingertip_f24,home_away_saveshands_f24,home_away_savesparrieddanger_f24,home_away_savesparriedsafe_f24,home_away_savesreaching_f24,home_away_savesstanding_f24,home_away_savesstooping_f24,home_away_shotsclearedofflineinsidebox_f24,home_away_shotsclearedofflineoutsidebox_f24,home_away_accurate_back_zone_pass,home_away_accurate_corners_intobox,home_away_accurate_cross,home_away_accurate_keeper_throws,home_away_accurate_through_ball,home_away_aerial_lost,home_away_aerial_won,home_away_att_hd_goal,home_away_att_hd_off_target,home_away_att_hd_target,home_away_att_ibox_blocked,home_away_att_ibox_goal,home_away_att_ibox_off_target,home_away_att_ibox_target,home_away_att_obox_blocked,home_away_att_obox_goal,home_away_att_obox_off_target,home_away_att_obox_target,home_away_att_pen_goal,home_away_att_pen_off_target,home_away_att_pen_target,home_away_ball_recovery,home_away_challenge_lost,home_away_clean_sheet,home_away_clean_sheet_away,home_away_clean_sheet_part,home_away_clearance_off_line,home_away_direct_goals,home_away_drops,home_away_error_lead_to_goal,home_away_error_lead_to_shot,home_away_failed_back_zone_pass,home_away_failed_corners_intobox,home_away_failed_cross,home_away_failed_fwd_zone_pass,home_away_failed_keeper_throws,home_away_fouls,home_away_gk_smother,home_away_goal_assist,home_away_goal_assist_openplay,home_away_goals_conceded_ibox,home_away_goals_conceded_obox,home_away_good_high_claim,home_away_hand_ball,home_away_head_clearance,home_away_hit_woodwork,home_away_interception,home_away_key_passes,home_away_key_passes_open_play,home_away_last_man_tackle,home_away_long_pass_own_to_opp,home_away_long_pass_own_to_opp_success,home_away_lost_contest,home_away_lost_corners,home_away_lost_tackle,home_away_nohead_clearance,home_away_own_goals,home_away_penalty_conceded,home_away_penalty_save,home_away_penalty_won,home_away_punches,home_away_red_card,home_away_saved_ibox,home_away_saved_obox,home_away_second_yellow,home_away_shield_ball_oop,home_away_six_yard_block,home_away_successful_final_third_passes,home_away_total_attacking_pass,home_away_total_fwd_zone_pass,home_away_total_offside,home_away_was_fouled,home_away_won_contest,home_away_won_corners,home_away_won_tackle,home_away_yellow_card,home_home_equalizer_goal,home_home_matchwinner_goal,home_home_outofplayconceded_f24,home_home_savesbody_f24,home_home_savescaught_f24,home_home_savescollected_f24,home_home_savesdiving_f24,home_home_savesfeet_f24,home_home_savesfingertip_f24,home_home_saveshands_f24,home_home_savesparrieddanger_f24,home_home_savesparriedsafe_f24,home_home_savesreaching_f24,home_home_savesstanding_f24,home_home_savesstooping_f24,home_home_shotsclearedofflineinsidebox_f24,home_home_shotsclearedofflineoutsidebox_f24,home_home_accurate_back_zone_pass,home_home_accurate_corners_intobox,home_home_accurate_cross,home_home_accurate_keeper_throws,home_home_accurate_through_ball,home_home_aerial_lost,home_home_aerial_won,home_home_att_hd_goal,home_home_att_hd_off_target,home_home_att_hd_target,home_home_att_ibox_blocked,home_home_att_ibox_goal,home_home_att_ibox_off_target,home_home_att_ibox_target,home_home_att_obox_blocked,home_home_att_obox_goal,home_home_att_obox_off_target,home_home_att_obox_target,home_home_att_pen_goal,home_home_att_pen_off_target,home_home_att_pen_target,home_home_ball_recovery,home_home_challenge_lost,home_home_clean_sheet,home_home_clean_sheet_away,home_home_clean_sheet_part,home_home_clearance_off_line,home_home_direct_goals,home_home_drops,home_home_error_lead_to_goal,home_home_error_lead_to_shot,home_home_failed_back_zone_pass,home_home_failed_corners_intobox,home_home_failed_cross,home_home_failed_fwd_zone_pass,home_home_failed_keeper_thr

In [14]:
sum_home_team_matches['score_home']

0.0

In [54]:
filepath = 'data/matches.csv'
matches = pd.read_csv(filepath)
matches

,match_id,competition_id,season_id,date,home_team_name,away_team_name,score_home,score_away,half_score_home,half_score_away,away_equalizer_goal,away_matchwinner_goal,away_outofplayconceded_f24,away_savesbody_f24,away_savescaught_f24,away_savescollected_f24,away_savesdiving_f24,away_savesfeet_f24,away_savesfingertip_f24,away_saveshands_f24,away_savesparrieddanger_f24,away_savesparriedsafe_f24,away_savesreaching_f24,away_savesstanding_f24,away_savesstooping_f24,away_shotsclearedofflineinsidebox_f24,away_shotsclearedofflineoutsidebox_f24,away_accurate_back_zone_pass,away_accurate_corners_intobox,away_accurate_cross,away_accurate_keeper_throws,away_accurate_through_ball,away_aerial_lost,away_aerial_won,away_att_hd_goal,away_att_hd_off_target,away_att_hd_target,away_att_ibox_blocked,away_att_ibox_goal,away_att_ibox_off_target,away_att_ibox_target,away_att_obox_blocked,away_att_obox_goal,away_att_obox_off_target,away_att_obox_target,away_att_pen_goal,away_att_pen_off_target,away_att_pen_target,away_ball_recovery,away_challenge_lost,away_clean_sheet,away_clean_sheet_away,away_clean_sheet_part,away_clearance_off_line,away_direct_goals,away_drops,away_error_lead_to_goal,away_error_lead_to_shot,away_failed_back_zone_pass,away_failed_corners_intobox,away_failed_cross,away_failed_fwd_zone_pass,away_failed_keeper_throws,away_fouls,away_gk_smother,away_goal_assist,away_goal_assist_openplay,away_goals_conceded_ibox,away_goals_conceded_obox,away_good_high_claim,away_hand_ball,away_head_clearance,away_hit_woodwork,away_interception,away_key_passes,away_key_passes_open_play,away_last_man_tackle,away_long_pass_own_to_opp,away_long_pass_own_to_opp_success,away_lost_contest,away_lost_corners,away_lost_tackle,away_nohead_clearance,away_own_goals,away_penalty_conceded,away_penalty_save,away_penalty_won,away_punches,away_red_card,away_saved_ibox,away_saved_obox,away_second_yellow,away_shield_ball_oop,away_six_yard_block,away_successful_final_third_passes,away_total_attacking_pass,away_total_fwd_zone_pass,away_total_offside,away_was_fouled,away_won_contest,away_won_corners,away_won_tackle,away_yellow_card,home_equalizer_goal,home_matchwinner_goal,home_outofplayconceded_f24,home_savesbody_f24,home_savescaught_f24,home_savescollected_f24,home_savesdiving_f24,home_savesfeet_f24,home_savesfingertip_f24,home_saveshands_f24,home_savesparrieddanger_f24,home_savesparriedsafe_f24,home_savesreaching_f24,home_savesstanding_f24,home_savesstooping_f24,home_shotsclearedofflineinsidebox_f24,home_shotsclearedofflineoutsidebox_f24,home_accurate_back_zone_pass,home_accurate_corners_intobox,home_accurate_cross,home_accurate_keeper_throws,home_accurate_through_ball,home_aerial_lost,home_aerial_won,home_att_hd_goal,home_att_hd_off_target,home_att_hd_target,home_att_ibox_blocked,home_att_ibox_goal,home_att_ibox_off_target,home_att_ibox_target,home_att_obox_blocked,home_att_obox_goal,home_att_obox_off_target,home_att_obox_target,home_att_pen_goal,home_att_pen_off_target,home_att_pen_target,home_ball_recovery,home_challenge_lost,home_clean_sheet,home_clean_sheet_away,home_clean_sheet_part,home_clearance_off_line,home_direct_goals,home_drops,home_error_lead_to_goal,home_error_lead_to_shot,home_failed_back_zone_pass,home_failed_corners_intobox,home_failed_cross,home_failed_fwd_zone_pass,home_failed_keeper_throws,home_fouls,home_gk_smother,home_goal_assist,home_goal_assist_openplay,home_goals_conceded_ibox,home_goals_conceded_obox,home_good_high_claim,home_hand_ball,home_head_clearance,home_hit_woodwork,home_interception,home_key_passes,home_key_passes_open_play,home_last_man_tackle,home_long_pass_own_to_opp,home_long_pass_own_to_opp_success,home_lost_contest,home_lost_corners,home_lost_tackle,home_nohead_clearance,home_own_goals,home_penalty_conceded,home_penalty_save,home_penalty_won,home_punches,home_red_card,home_saved_ibox,home_saved_obox,home_second_yellow,home_shield_ball_oop,home_six_yard_block,home_successful_final_third_passes,home_total_attacking_pass,home_total_fwd_